In [1]:
%config Application.log_level="ERROR"
import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
logger=logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
import time
import json
import pandas as pd

In [3]:
def find_similar_items(item,likeness):
    matches=[]
    try:
        matches=matches+[ likeness.xs(item,level=0) ]
    except:
        pass
    try:
        matches=matches+[ likeness.xs(item,level=1) ]
    except:
        pass
    if len(matches)>0:      
        return pd.concat(matches).rename_axis('song').reset_index()
    else:
        return pd.DataFrame(columns=['song','counts'])
def give_me_recs(songs_lst,likeness,num=50):
    recs=pd.concat([find_similar_items(song,likeness) for song in songs_lst]).groupby('song')['counts'].sum()
    recs=recs.loc[~recs.index.isin(songs_lst)].sort_values(ascending=False).head(num).reset_index()
    return recs

In [13]:
bookmarks_file='./src/data/bookmarks.txt'
songs_file='./src/data/songs.txt'
#songs_file='./data/songs_enhanced.txt'
likeness_file='./src/data/likeness.txt'
bookmarks_file,songs_file,likeness_file

('./src/data/bookmarks.txt', './src/data/songs.txt', './src/data/likeness.txt')

In [6]:
%%time
with open(bookmarks_file,'r') as f:
    bookmarks_lst={row.strip().split(',',1)[0]:row.strip().split(',',1)[1].split(',') for row in f.readlines()}
with open(songs_file,'r') as f:
    songs={song.strip().split(',',1)[0]:json.loads(song.strip().split(',',1)[1]) for song in f.readlines()}
likeness=pd.read_csv(likeness_file,index_col=[0,1],names=['counts'])

CPU times: user 11.8 ms, sys: 3.95 ms, total: 15.7 ms
Wall time: 14.5 ms


In [7]:
user='saaaaaage'

In [9]:
%%time
if user in bookmarks_lst:
    recs=give_me_recs(bookmarks_lst[user],likeness,20)
    recs['title']=recs['song'].apply(lambda x: songs[x]['title'])
#    recs['rating']=recs['song'].apply(lambda x: songs[x]['overall_rating'] if 'overall_rating' in songs[x] else '')
    recs['curator']=recs['song'].apply(lambda x: songs[x]['curated_by'] if 'curated_by' in songs[x] else '') 
    print(time.strftime('%Y%m%d-%H%M%S',time.localtime())+' - Recommendations for {} ready!'.format(user))
else:
    print(time.strftime('%Y%m%d-%H%M%S',time.localtime())+' - Not enough info about user {} to make recommendations!'.format(user))

20211208-170749 - Recommendations for saaaaaage ready!
CPU times: user 87.9 ms, sys: 0 ns, total: 87.9 ms
Wall time: 84.7 ms


In [10]:
recs

,song,counts,title,curator
0,3038,138,Moves Like Jagger - Maroon 5 ft. Christina Agu...,skylerwallace
1,462,113,The Nights - Avicii,lebandit915
2,141,110,GANGNAM STYLE,teroc
3,5fc,109,Waiting For Love - Avicii,thelessergatsby
4,2087,95,Panic at the disco - The Greatest Showman reim...,tan
5,4adb,93,Deja Vu - Dave Rodgers (Initial D),curatorrecommended
6,7b68,92,First of the Year (Equinox) - Skrillex,curatorrecommended
7,1fef,90,Imagine Dragons - Believer (100k ver.) | 100k ...,tan
8,176c,86,24K Magic - Bruno Mars,curatorrecommended
9,849a,86,Darude Sandstorm,curatorrecommended


In [12]:
bookmarks=['3038','462','141']
recs=give_me_recs(bookmarks,likeness,20)
recs['title']=recs['song'].apply(lambda x: songs[x]['title'])
recs['curator']=recs['song'].apply(lambda x: songs[x]['curated_by'] if 'curated_by' in songs[x] else '') 
recs

,song,counts,title,curator
0,570,26,Mr. Blue Sky - Electric Light Orchestra,thelessergatsby
1,5fc,22,Waiting For Love - Avicii,thelessergatsby
2,4a35,21,[Anniversary] Animals (Radio Edit) - Martin Ga...,curatorrecommended
3,9d7,21,Back in Black - AC/DC,
4,5038,20,Uptown Funk - Mark Ronson,tan
5,4c6,19,Muse - Uprising,skykiwi
6,26f6,18,IGNITE (Ranked Version) [Sword Art Online Seas...,
7,8553,17,Blinding Lights - The Weeknd,curatorrecommended
8,849a,17,Darude Sandstorm,curatorrecommended
9,7b68,17,First of the Year (Equinox) - Skrillex,curatorrecommended
